In [43]:
import pandas as pd

In [44]:
df_cdi = pd.read_csv("CDI.csv")
df_cdi

,fonte,valor,data,dono,tag,tipo_transacao,comentario
0,rodrigo,24807.98,20230416,rodrigo,reservas,entrada,NaN
1,dayane,12480.07,20230416,dayane,robank_dayane,entrada,NaN
2,rodrigo,266.41,20230416,rodrigo,caixa_investimentos,entrada,NaN
3,dayane,-300.00,20230503,dayane,robank_dayane,saida,saque velorio Bruno
4,rendimento,50.70,20230619,dayane,robank_dayane,entrada,rendimento
5,rodrigo,6923.78,20230526,rodrigo,reservas,entrada,deposito
6,rodrigo,3100.00,20230526,rodrigo,caixa_investimentos,entrada,deposito
7,rendimento,454.66,20230619,rodrigo,reservas,entrada,rendimento
8,rodrigo,-3000.00,20230619,rodrigo,caixa_investimentos,saida,compra de dolar
9,rendimento,23.01,20230619,rodrigo,caixa_investimentos,entrada,rendimento


In [45]:
df_moedas = pd.read_csv("moedas.csv")
df_moedas

,fonte,ativo,valor,preco_em_real,custo,tipo_transacao,data,dono,tag
0,rodrigo,dolar,1995.22,4.993,37.86,compra,20230412,rodrigo,NaN
1,dayane,dolar,946.63,5.281,18.93,compra,20221223,dayane,robank_dayane
2,rodrigo,dolar,946.63,5.281,18.93,compra,20221223,rodrigo,NaN
3,rodrigo,dolar,952.53,5.249,18.93,compra,20221107,rodrigo,NaN
4,rodrigo,dolar,619.96,4.820,11.36,compra,20230619,rodrigo,NaN
5,rodrigo,dolar,-2046.40,0.000,3.00,venda,20230620,rodrigo,NaN
6,dayane,dolar,-511.60,0.000,0.75,venda,20230620,dayane,robank_dayane


In [46]:
df_proventos = pd.read_csv("proventos.csv")
df_proventos

,fonte,quantiade,valor,data_negociacao,tipo_de_ordem,IRRF,moeda,dono
0,ITUB4,100,1.50,20230403,juros_de_capital_proprio,0.00,real,rodrigo
1,ITSA4,1760,35.20,20230403,juros_de_capital_proprio,0.00,real,rodrigo
2,BBAS3,100,29.92,20230331,juros_de_capital_proprio,0.00,real,rodrigo
3,ITUB4,100,41.32,20230310,juros_de_capital_proprio,0.00,real,rodrigo
4,ITSA4,1760,210.94,20230310,juros_de_capital_proprio,0.00,real,rodrigo
5,BBAS3,100,0.40,20230303,rendimentos,0.11,real,rodrigo
6,BBAS3,100,0.98,20230303,rendimentos,0.28,real,rodrigo
7,BBAS3,100,48.75,20230303,juros_de_capital_proprio,0.00,real,rodrigo
8,BBAS3,100,23.54,20230303,dividendos,0.00,real,rodrigo
9,ITUB4,100,1.50,20230301,juros_de_capital_proprio,0.00,real,rodrigo


In [47]:
df_trades = pd.read_csv("trades.csv")
df_trades

,ativo,quantidade,preco,data_negociacao,tipo_de_ordem,fonte,dono,custo,moeda
0,IRBR3,-967,0.00,20230125,agrupamento,IRBR3,rodrigo,0.00,real
1,ITSA4,600,8.85,20221111,compra,rodrigo,rodrigo,0.00,real
2,ITSA4,500,8.85,20221111,compra,rodrigo,rodrigo,0.00,real
3,ITSA4,60,13.65,20221111,bonus_em_acoes,ITSA4,rodrigo,0.00,real
4,MGLU3,100,3.51,20221111,compra,rodrigo,rodrigo,0.00,real
5,ITSA4,400,9.84,20221110,compra,rodrigo,rodrigo,0.00,real
6,BBAS3,100,36.40,20221110,compra,rodrigo,rodrigo,0.00,real
7,ITUB4,100,27.12,20221110,compra,rodrigo,rodrigo,0.00,real
8,ITSA4,200,10.50,20221107,compra,rodrigo,rodrigo,0.00,real
9,CMIG4,200,11.65,20221107,compra,rodrigo,rodrigo,0.00,real


In [5]:
from analytics import Analytics

calculadora =  Analytics()

print('-------- Rodrigo --------')
res = calculadora.calcular_patrimonio('rodrigo')
print(f"Patrimonio total: {res:,.2f}\n")

print('-------- Dayane --------')
res = calculadora.calcular_patrimonio('dayane')
print(f"Patrimonio total: {res:,.2f}\n")

-------- Rodrigo --------
Real: 45,317.31
Dolar: 2,467.94 (13,080.08)
Ativos Real: 32,975.44
Ativos Dolar: 2,090.80 (11,081.24)
Patrimonio total: 102,454.07

-------- Dayane --------
Real: 12,230.77
Dolar: 435.03 (2,305.66)
Ativos Real: 0.00
Ativos Dolar: 522.70 (2,770.31)
Patrimonio total: 17,306.74

